This tutorial deals with reading a SEG-Y data set and __creating an SU file__. Please make sure you have downloaded the test data as outlined in the file "examples/data/README".

Let's start by importing some packages (including the seisio package itself) and setting up the logging config.

In [1]:
import logging

import sys
sys.path.insert(1, '../')

In [2]:
import seisio

In [3]:
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s', force=True)
log = logging.getLogger("main")

Let's read the SEG-Y file "Line_001.sgy" and convert the file to Seismic Unix format, storing the data in "Line_001.su":

In [4]:
infile = "data/Line_001.sgy"   # SEGY file, IBM floats, big endian
outfile = "data/Line_001.su"   # SU file, IEEE floats, little endian

First, we create a seisio object to read the SEG-Y file. All relevant parameters will be determined automatically (see previous tutorial for details):

In [5]:
sin = seisio.input(infile)

INFO: Input file: data/Line_001.sgy
INFO: SEG-Y textual file header encoding looks to be 'EBCDIC' (best guess).
INFO: Input file endianess looks to be '>' (best guess).
INFO: SEG-Y revision: original SEG-Y conforming to 1975 standard.
INFO: Data sample format: 4-byte IBM floating-point.
INFO: Number of additional textual header records: 0.
INFO: Number of trailer stanza records: 0.
INFO: SEG-Y trace header extension 1 is not present.
INFO: Number of user-defined trace headers: 0
INFO: Byte offset of first trace relative to start of file: 3600 bytes.
INFO: Number of samples per data trace: 1501.
INFO: Sampling interval: 2000 (unit as per SEG-Y standard).
INFO: Number of data traces in file: 71284.
INFO: Delay (on first trace): 0 (unit as per SEG-Y standard).


Now we create another seisio object for the output in SU format. We would like to store the data in little-endian byte order. As we open the file in "write" mode, any existing file of the same name will be truncated (i.e., overwritten) and you get a corresponding warning - this happens, for instance, if you run the jupyter cell several times.

In [6]:
sout = seisio.output(outfile, ns=sin.ns, mode="w", endian="<")

INFO: Output file: data/Line_001.su
INFO: Assuming fixed-length traces for SU data.
INFO: Output number of samples per data trace: 1501.
INFO: Output data sample format: 4-byte IEEE floating-point.
INFO: Output file endianess set to '<'.
INFO: Byte offset of first trace relative to start of file: 0 bytes.


Let's convert the file in steps rather than reading the whole file at once. We would like to remap the input trace header mnemonic `fldr` to the trace header mnemonic `cdp` in the output (this would not normally make sense, we just do it to demonstrate the remapping itself). For each batch we read and convert, we also set the `scalel` and `scalco` trace headers (providing information on elevation and coordinate scalers) to zero. As SEG-Y has different trace headers compared to SU, you will see a few warnings that certain headers need to be dropped because there is no equivalent header mnemonic in the SU header definition. This could be avoided by remapping the corresponding headers to trace header mnemonics available in the SU world, or providing a non-standard SU trace header definition table (JSON file) when creating the output object.

In [7]:
header_map = {"fldr": "cdp"}

for batch in sin.batches(batch_size=30000):
    batch["scalel"] = 0   # set 'scalel' header to zero
    batch["scalco"] = 0   # set 'scalco' header to zero
    sout.write_traces(traces=batch, remap=header_map)

INFO: Reading 30000 trace(s) from disk starting at trace 0...
INFO: Remapping mnemonic 'fldr' to 'cdp'.
INFO: Swapping bytes of output data.
INFO: Writing 30000 trace(s) to disk...
INFO: Reading 30000 trace(s) from disk starting at trace 30000...
INFO: Remapping mnemonic 'fldr' to 'cdp'.
INFO: Swapping bytes of output data.
INFO: Writing 30000 trace(s) to disk...
INFO: Reading 11284 trace(s) from disk starting at trace 60000...
INFO: Remapping mnemonic 'fldr' to 'cdp'.
INFO: Swapping bytes of output data.
INFO: Writing 11284 trace(s) to disk...


As we have read data in big-endian byte order, the data need to be swapped on output.

You can now run the program "surange" on the SU data file and the output should look as follows (provided your SU installation was compiled to deal with data in native endianess and you are on a host that uses little-endian byte order):

```
$> surange < data/Line_001.su
71284 traces:
tracl    1 71284 (1 - 71284)
tracr    1 71284 (1 - 71284)
fldr     231 481 (231 - 481)
tracf    -1 282 (-1 - 282)
ep       32 282 (32 - 282)
cdp      231 481 (231 - 481)
cdpt     1 284 (1 - 284)
trid     0 1 (0 - 1)
nhs      1
counit   3
ns       1501
dt       2000
gain     3
igc      1
afilf    207
afils    298
hcf      207
hcs      298
year     1998
$>
```

As can be seen, the `fldr` header was cloned and the `cdp` header is now present in the SU file (obviously, these numbers are no real CMP numbers, this mapping was just performed to demonstrate the feature). The `scalel` and `scalco` headers do not show up, i.e., they have been set to zero as outlined above.
